In [6]:
!pip install mysqlclient

In [36]:
import pandas as pd
import requests as req
from citipy import citipy 
import matplotlib.pyplot as plt
import seaborn as sns
import random
import MySQLdb
import time as time
import openweathermapy.core as owm
pd.options.mode.chained_assignment = None
# from config.py import API_key
# key=API_key
key="c2747a1a47120dfe77da0d1c5bfd583b"
import pymysql, os
pymysql.install_as_MySQLdb()



In [37]:
# load both csv fle into dataframe
parks_data = r"parks.csv"
species_data= r"species.csv"

In [38]:
parks_dataframe= pd.read_csv(parks_data,encoding = "ISO-8859-1")
parks_dataframe= parks_dataframe.drop_duplicates()
parks_dataframe.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [39]:
species_dataframe= pd.read_csv(species_data,encoding = "ISO-8859-1")
species_dataframe= species_dataframe.drop_duplicates()
species_dataframe.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


In [40]:
len(parks_dataframe)

56

In [ ]:
#use openwesthermaap api caalls to have weather information against every entry in the dataframe corresponding to parks
max_temp = []
min_temp = []
temp = []
humidity =[]
clouds = []
wind = []

valid_data_counter = 0

url = "https://api.openweathermap.org/data/2.5/weather?lat="
units = "imperial"

#iterate through the rows to pull data from the api 
for index, row in parks_dataframe.iterrows():
    
        lat = row["Latitude"]
        long = row["Longitude"]
        target_url = url+str(lat)+"&lon="+str(long)+"&appid="+key+"&units="+units
        print(target_url)

        info = req.get(target_url).json()
        if (info['cod'] != "404"):

                valid_data_counter +=1
                temp.append(info['main']['temp'])
                max_temp.append(info['main']['temp_max'])
                min_temp.append(info['main']['temp_max'])
                humidity.append(info['main']['humidity'])
                clouds.append(info['clouds']['all'])
                wind.append(info['wind']['speed'])
                print("Data Fetched and Processed: Still Remaining: " + str(len(parks_dataframe)-valid_data_counter))

                time.sleep(1)
                print("__________________________________________________________________________________________")

        else:
                temp.append("No Response from API")
                max_temp.append("No Response from API")
                min_temp.append("No Response from API")
                humidity.append("No Response from API")
                clouds.append("No Response from API")
                wind.append("No Response from API")

                print("No Response from API: Still Remaining: " + str(len(parks_dataframe)-valid_data_counter))
                print("__________________________________________________________________________________________")
    
          
      

    #print("Total "+str(counter)+ " cities traversed for having "+ str(valid_data_counter) +" valid data.")
print("=============================================SUMMARY===================================================")
print("Total "+ str(valid_data_counter) +" parks traversed for having "+ str(len(parks_dataframe)) +" valid data.")
print("=============================================FINISHED==================================================")
#assign the values from the arrays to the paarks data frame
parks_dataframe["Temperature"]=temp
parks_dataframe["Max. Temperature"]=max_temp
parks_dataframe["Min. Temperature"]=min_temp
parks_dataframe["Humidity"]=humidity
parks_dataframe["Clouds"]=clouds
parks_dataframe["Wind Speed"]=wind

https://api.openweathermap.org/data/2.5/weather?lat=44.35&lon=-68.21&appid=c2747a1a47120dfe77da0d1c5bfd583b&units=imperial
Data Fetched and Processed: Still Remaining: 55
__________________________________________________________________________________________
https://api.openweathermap.org/data/2.5/weather?lat=38.68&lon=-109.57&appid=c2747a1a47120dfe77da0d1c5bfd583b&units=imperial
Data Fetched and Processed: Still Remaining: 54
__________________________________________________________________________________________
https://api.openweathermap.org/data/2.5/weather?lat=43.75&lon=-102.5&appid=c2747a1a47120dfe77da0d1c5bfd583b&units=imperial
Data Fetched and Processed: Still Remaining: 53
__________________________________________________________________________________________
https://api.openweathermap.org/data/2.5/weather?lat=29.25&lon=-103.25&appid=c2747a1a47120dfe77da0d1c5bfd583b&units=imperial
Data Fetched and Processed: Still Remaining: 52
_________________________________________

Data Fetched and Processed: Still Remaining: 24


In [ ]:
parks_dataframe = parks_dataframe[parks_dataframe['Temperature']!='No Response from API'] 
parks_dataframe.head()

In [ ]:
# slicing the no of records from the species.csv file to make it less in size to insert into database table
filename = r"species.csv"
import random
p = 0.01  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
species_dataframe = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

In [ ]:
#Now to create a foriegn key in the species dataframe we can split the species dataframe at '-' and get the park code and 
#create a new column in the species dataframe with the naame 'Park Code' which will servee as the foriegn key
#NOTE: 'Park Code' column in the park datafraame may be treated as the primary key

parkCode = []
for index, row in species_dataframe.iterrows():
    parkCode.append(row["Species ID"].split('-')[0])
#Assign the values of the array to a new column in dataframe
species_dataframe["Park Code"]=parkCode
#Rearrange the columns by setting the newly added column at the first place
species_dataframe = species_dataframe[["Park Code","Species ID","Park Name","Category","Order","Family","Scientific Name","Common Names","Record Status","Occurrence","Nativeness","Abundance","Seasonality","Conservation Status","Unnamed: 13"]]
species_dataframe.head()

In [ ]:
#Cleaning of both the data frames.
parks_dataframe = parks_dataframe.dropna(how='all')
species_dataframe = species_dataframe.dropna(how='all')
species_dataframe = species_dataframe.fillna('')

In [ ]:
# Check database connection
import mysql.connector
db_connection = mysql.connector.connect(
  host="localhost",
  user="root",
  PASSWD = os.getenv("LOCAL_MYSQL_PASS")
)
print(db_connection)

In [ ]:
# creating database_cursor to perform SQL operation
db_cursor = db_connection.cursor()
# executing cursor with execute method and pass SQL query
db_cursor.execute("create DATABASE ETLWork")

In [ ]:
# get list of all databases
db_cursor.execute("SHOW DATABASES")
#print all databases
print("Print all Databases")
for db in db_cursor:
    print(db)

In [ ]:
# Create tables
import sqlalchemy
database_username = 'root'
database_password = os.getenv("LOCAL_MYSQL_PASS")
database_ip       = 'localhost'
database_name     = 'etlwork'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
parks_dataframe.to_sql(con=database_connection, name='parks', if_exists='replace')

In [ ]:
species_dataframe.to_sql(con=database_connection, name='species', if_exists='replace')

In [ ]:
# Change default datatype of column to be made primary and foriegn in both the table 
# Note: There comes a blob/text error when taken up with default text values, so we are doing it by converting from text to varchar(4)
database_connection.execute("ALTER TABLE `parks` MODIFY `Park Code` VARCHAR(4);")
database_connection.execute("ALTER TABLE `species` MODIFY `Park Code` VARCHAR(4);")

In [ ]:
# Make primary key in parks table
database_connection.execute("ALTER TABLE `parks` ADD PRIMARY KEY (`Park Code`);")

In [ ]:
# Make foreign key in Species table referencing to parks
database_connection.execute("ALTER TABLE `species` ADD FOREIGN KEY (`Park Code`) REFERENCES parks(`Park Code`);")

In [ ]:
# Now assign any query to the variable named sql below annd run ro get the queryresult
sql = "select * from species"
queryResult = pd.read_sql(sql, database_connection)
queryResult

In [ ]:
# To find all the results from species table which falls under the category of ''Bird''''
sql = "select * from species where Category='Bird'"
queryResult = pd.read_sql(sql, database_connection)
queryResult